## chatrwkv 사용

In [1]:
# https://github.com/BlinkDL/ChatRWKV
!pip install rwkv
!pip install tokenizers

     -------------------------------------- 395.3/395.3 kB 8.2 MB/s eta 0:00:00


In [2]:
# 모델다운로드: https://huggingface.co/BlinkDL
# https://huggingface.co/BlinkDL/rwkv-4-pile-169m/tree/main

!wget https://huggingface.co/BlinkDL/rwkv-4-pile-169m/resolve/main/RWKV-4-Pile-169M-20220807-8023.pth 

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [ ]:
# 토크나이저 다운
# !rm -rf ChatRWKV
!git clone https://github.com/BlinkDL/ChatRWKV
!mv ChatRWKV/20B_tokenizer.json .

Cloning into 'ChatRWKV'...
remote: Enumerating objects: 1110, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1110 (delta 0), reused 2 (delta 0), pack-reused 1103
Receiving objects: 100% (1110/1110), 4.78 MiB | 15.80 MiB/s, done.
Resolving deltas: 100% (634/634), done.


In [ ]:
# https://huggingface.co/spaces/BlinkDL/ChatRWKV-gradio
# https://github.com/BlinkDL/ChatRWKV/blob/main/RWKV_in_150_lines.py
# https://pypi.org/project/rwkv/
from rwkv.model import RWKV
from rwkv.utils import PIPELINE, PIPELINE_ARGS

def my_print(s):
    print(s, end='', flush=True)

model_path = 'RWKV-4-Pile-169M-20220807-8023'
model = RWKV(model=model_path, strategy='cuda fp16 *32 -> cpu fp32') # strategy='cpu fp32'
# https://huggingface.co/docs/transformers/main_classes/pipelines
pipeline = PIPELINE(model, "20B_tokenizer.json") 

# For alpha_frequency and alpha_presence, see "Frequency and presence penalties":
# https://platform.openai.com/docs/api-reference/parameter-details

args = PIPELINE_ARGS(temperature = 1.0, top_p = 0.7,
                     alpha_frequency = 0.25,
                     alpha_presence = 0.25,
                     token_ban = [0], # ban the generation of some tokens
                     token_stop = []) # stop generation whenever you see any token here

In [ ]:
prompt = "Do you know chatgpt?" # 
print(prompt, end='')

# 생성하기
result = pipeline.generate(prompt, token_count=200, args=args, callback=my_print)
print(result)
print('\n')

In [ ]:
# 모델 logit 출력하기
out, state = model.forward([187, 510, 1563, 310, 247], None)
print(out.detach().cpu().numpy())                   # get logits
out, state = model.forward([187, 510], None)
out, state = model.forward([1563], state)           # RNN has state (use deepcopy to clone states)
out, state = model.forward([310, 247], state)
print(out.detach().cpu().numpy())                   # same result as above
print('\n')

In [ ]:
# 학습을 하고 싶다면?
https://github.com/resloved/RWKV-notebooks

## chatGPT 사용

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 3.2 MB/s eta 0:00:00


In [ ]:
import openai
openapi_key = 'your_key'
openai.api_key = openapi_key

In [ ]:
model="gpt-3.5-turbo"
query = "How to learn English?"

In [ ]:
"""
    ref: https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide
    ref: https://wooiljeong.github.io/python/chatgpt-api/

    messages: 생성하려는 대화의 히스토리를 정의하는데 사용됩니다. 대화에 참여하는 여러 역할(‘system(시스템)’, ‘assistant(도우미)’, ‘user(사용자)’)과 메시지 내용을 설정할 수 있습니다. 
    보통 대화는 먼저 시스템 메시지로 형식을 정의합니다. 그 다음 사용자와 도우미의 메시지를 번갈아가며 정의합니다.

    시스템 메시지: ‘You are a helpful assistant.’와 같은 메시지로 도우미에게 지시할 수 있습니다. 시스템이 챗봇에게 일종의 역할을 부여한다고 볼 수 있습니다.
    사용자 메시지: 도우미에게 직접 전달하는 내용입니다.
    도우미 메시지: 이전에 응답했던 결과를 저장해 대화의 흐름을 유지할 수 있도록 설정할 수 있습니다.
"""
messages = [
        {"role": "user", "content": query}
]

messages1 = [
        {"role": "system", "content": "You are an assistant that speaks like child."},
        {"role": "user", "content": query},
]

completion = openai.ChatCompletion.create(
    model=model,
    messages=messages
)

completion1 = openai.ChatCompletion.create(
    model=model,
    messages=messages1
)

In [ ]:
# print(completion)

In [ ]:
response = completion['choices'][0]['message']['content']
print(response)

# 1. Start by setting goals: Determine why you want to learn English and what your goals are. Do you want to improve your speaking skills, prepare for an exam, or improve your job prospects? Setting goals can help you focus your learning efforts.

# 2. Practice regularly: Set aside some time every day or every week to practice English. This can include watching videos or movies, reading books or articles, practicing speaking with native speakers, writing paragraphs, or listening to English podcasts.

# 3. Immerse yourself in English: Surround yourself with English as much as possible. Watch English-language TV shows and movies, listen to English music and radio, read English newspapers or books, and interact with English-speaking people as much as possible.

# 4. Take classes: If you prefer a more structured approach to learning English, consider enrolling in classes at a language school or taking online courses. This can help you build a solid foundation in grammar, vocabulary, and conversation skills.

# 5. Use technology to your advantage: There are many apps and online tools that can help you learn English. Duolingo, Babbel, and Busuu are popular language learning apps, and websites like BBC Learning English and ESLPod offer free resources.

# 6. Practice speaking: Practice speaking with other English learners or native speakers. You can join language exchange groups or language exchange apps like iTalki, HelloTalk, or Tandem to find conversation partners.

# 7. Don't be afraid to make mistakes: Learning a language involves making mistakes. Don't be afraid to practice speaking or writing in English, even if you make mistakes. Remember that mistakes are a natural part of the learning process.

As an AI language model, I can provide you few tips on how to learn English:

1. Start with the basics: Learn the English alphabet, grammar rules, and simple vocabulary. Setting a strong foundation is important to build upon.

2. Watch and listen to English content: Watch TV shows, movies, news, and listen to English language songs, podcasts, or radio. This will help you train your ears to understand different accents, increase your vocabulary, and practice pronunciation.

3. Have conversations in English: Find native speakers or language exchange partners to practice speaking and listening skills. Conversational practice is immensely helpful to improve fluency and clarity.

4. Read English books and articles: Start with beginner-level books and gradually move to more advanced ones. This will help you develop reading comprehension, writing, and critical thinking skills.

5. Take online English courses: There are numerous online courses available for different levels of proficiency. The

In [ ]:
response = completion1['choices'][0]['message']['content']
print(response)

# Oh wow! Learning English is super duper fun! Here are some tips to help you learn:

# 1. Read English books or articles out loud to improve your pronunciation and fluency.

# 2. Watch English films or TV shows with subtitles to help you understand and pick up new vocabulary.

# 3. Practice speaking with native English speakers, either in person or online, to improve your grammar and speaking skills.

# 4. Listen to English music and try to sing along to improve your listening and speaking skills.

# 5. Make flashcards with English words and their meanings and review them often to improve your vocabulary.

# 6. Find a language exchange partner to practice speaking with on a regular basis.

# Remember, practice makes perfect! Good luck with your English learning adventure!

Learning English can be so much fun! To start learning English, you can listen to songs and watch cartoons that are in English. You can also practice speaking English with friends or using language learning apps. Reading books and practicing writing in English can help too. Remember, it's important to practice every day and not be afraid to make mistakes. Keep trying and you will get better and better at English!


In [ ]:
query = "괌으로 여행가려고 하는데, 준비할 물품좀 알려줘"

messages = [
        {"role": "user", "content": query}
]


completion = openai.ChatCompletion.create(
    model=model,
    messages=messages
)

In [ ]:
response = completion['choices'][0]['message']['content']
print(response)

한국과 같은 문화권이므로 기본적인 준비물품은 크게 다르지 않겠지만, 다음과 같은 물품들을 추천드립니다.

1. 여권
2. 항공권
3. 신용카드 또는 현금
4. 전기 어댑터 (미국식 110V 전기 플러그로 변환용 어댑터)
5. 선크림 및 모자 등의 일회용품
6. 비행기 내부에서 사용할 수 있는 책, 음악 플레이어, 스마트폰, 태블릿 등 기기
7. 인삼, 홍삼 등의 건강보조 식품
8. 다목적 의약품 (해열제, 소화제 등)
9. 수영복, 비치웨어 등의 해변용 의류
10. 카메라나 셀카봉 등의 기록용품

괌의 날씨는 평균 온도가 27도 정도이므로, 겉옷이나 단도착용에 유의해주세요. 또한, 여행을 갈 때 반드시 1회용 마스크를 챙겨 놓는 것이 좋습니다.


In [ ]:
# 사용: ref: https://platform.openai.com/account/usage

## alphaca-lora

In [ ]:
# https://github.com/tatsu-lab/stanford_alpaca
!git clone https://github.com/tloen/alpaca-lora

In [ ]:
# !pip install -r alpaca-lora/requirements.txt
!pip install -r requirements.txt

In [ ]:
!mv alpaca-lora/* .

In [ ]:
!python generate.py \
    --load_8bit \
    --base_model 'decapoda-research/llama-7b-hf' \
    --lora_weights 'tloen/alpaca-lora-7b' \
    --share_gradio True

In [ ]:
# https://colab.research.google.com/drive/1eWAmesrW99p7e1nah5bipn0zikMb8XYC

In [ ]:
# https://github.com/huggingface/diffusers/issues/1207
# conda install cudatoolkit

import torch
from peft import PeftModel
import transformers
import pdb
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"    


device = torch.device('cuda')
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map= "auto",
)

model = PeftModel.from_pretrained(
    model, "tloen/alpaca-lora-7b", torch_dtype=torch.float16
).to(device)
# model = PeftModel.from_pretrained(
#     model, "tloen/alpaca-lora-7b", torch_dtype=torch.float16, device_map={'': 0}
# )    

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
trans_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(device)
trans_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")  



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-1otmqekxijwxp --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true'), PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=h

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [ ]:
def translate(text, src, tgt):
    trans_tokenizer.src_lang = src
    encoded = trans_tokenizer(text, return_tensors="pt").to(device)
    generated_tokens = trans_model.generate(**encoded, forced_bos_token_id=trans_tokenizer.get_lang_id(tgt))
    result = trans_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return result
    
    
def evaluate(
    instruction,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=2, # 4
    input=None,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=2048,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()
    
def generate_fascampus(instruction, ko_query):
    print("지침:", instruction)
    
    print("질의:", ko_query)
    en_query = translate(ko_query, 'ko', 'en')
    
    en_result = evaluate(instruction=instruction, num_beams=2, input=en_query)
    print("영어응답:", en_result)
    ko_result = translate(en_result, 'en', 'ko')
    print("한글응답:", ko_result)
    print()
    return en_result, ko_result

In [ ]:
find_num = 5
instruction = f"{find_num} ways to study"

while True:
    ko_query = input("공부할 질의: ")
    if ko_query == 'exit':
        break
    en_result, ko_result = generate_fascampus(instruction, ko_query)

공부할 질의: 영어
지침: 5 ways to study
질의: 영어


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


영어응답: Five ways to study English are: 
1. Read books and articles related to English. 
2. Listen to podcasts and audiobooks related to English. 
3. Watch movies and TV shows in English. 
4. Practice speaking English with a native speaker. 
5. Take an English class or online course.


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


한글응답: 영어를 공부하는 다섯 가지 방법은 다음과 같습니다 : 1. 영어와 관련된 책과 기사를 읽으십시오. 2. 영어와 관련된 팟캐스트와 오디오 책을 듣으십시오. 3. 영어로 영화와 TV 쇼를 보십시오. 4. 원주민 연설자와 함께 영어를 연습하십시오. 5. 영어 수업이나 온라인 코스를 취하십시오.

공부할 질의: 수학 연구
지침: 5 ways to study
질의: 수학 연구
영어응답: Five ways to study for Mathematical Studies are:
1. Read the textbook and take notes.
2. Make flashcards to help memorize key concepts.
3. Practice problem-solving exercises.
4. Take practice tests.
5. Seek help from a teacher or tutor.
한글응답: 수학 연구를 위해 공부하는 다섯 가지 방법은 다음과 같습니다 : 1. 책을 읽고 노트를 가져 가십시오. 2. 핵심 개념을 기억하는 데 도움이되는 플래시 카드를 만드십시오. 3. 문제 해결 연습을 실천하십시오. 4. 실습 테스트를 수행하십시오. 5. 교사 또는 교사의 도움을 구하십시오.



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/ipykernel/kernelbase.py:851 in raw_input                  │
│                                                                                                  │
│   848 │   │   │   raise StdinNotImplementedError(                                                │
│   849 │   │   │   │   "raw_input was called, but this frontend does not support input requests   │
│   850 │   │   │   )                                                                              │
│ ❱ 851 │   │   return self._input_request(str(prompt),                                            │
│   852 │   │   │   self._parent_ident,                                                            │
│   853 │   │   │   self._parent_header,                                                           │
│   854 │   │   │   password=False,                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/ipykernel/kernelbase.py:895 in _input_request             │
│                                                                                                  │
│   892 │   │   │   │   │   │   break                                                              │
│   893 │   │   │   except KeyboardInterrupt:                                                      │
│   894 │   │   │   │   # re-raise KeyboardInterrupt, to truncate traceback                        │
│ ❱ 895 │   │   │   │   raise KeyboardInterrupt("Interrupted by user") from None                   │
│   896 │   │   │   except Exception as e:                                                         │
│   897 │   │   │   │   self.log.warning("Invalid Message:", exc_info=True)                        │
│   898                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt: Interrupted by user

## opt 사용

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch

model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m", torch_dtype=torch.float16).cuda() # opt-30b # 175b

# the fast tokenizer currently does not work correctly
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m", use_fast=False)

In [ ]:
prompt = "I want to learn Enlgish."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

# set_seed(32)
generated_ids = model.generate(input_ids, do_sample=True)

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [ ]:
## llama

## llama 사용하기

In [ ]:
# https://github.com/shawwn/llama-dl?fbclid=IwAR21gvRDtH38tCgS1iW-gU-_CEVLkB7N-rIg-LfVaDvCTkQ1e6c450SZty8
!curl -o- https://raw.githubusercontent.com/shawwn/llama-dl/56f50b96072f42fb2520b1ad5a1d6ef30351f23c/llama.sh | bash

## PEFT 사용하기

In [ ]:
!pip install transformers
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_model, LoraConfig, TaskType
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

In [ ]:
# https://github.com/huggingface/peft/blob/main/src/peft/tuners/lora.py
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
# output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282

In [ ]:
## 출력할 모델 파타리터 함수
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
## 한국어 모델도 당연히 비슷하게 지원
## https://huggingface.co/skt/ko-gpt-trinity-1.2B-v0.5
from transformers import AutoModel
model_name_or_path = "skt/ko-gpt-trinity-1.2B-v0.5" # "bigscience/mt0-large"
tokenizer_name_or_path = "skt/ko-gpt-trinity-1.2B-v0.5" # "bigscience/mt0-large"

model = AutoModel.from_pretrained(model_name_or_path)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
peft_config = LoraConfig(
    task_type="FastCampus", inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
print_trainable_parameters(model)

In [ ]:
# 학습과정은 기존하고 똑같이 진행
model.save_pretrained("output_dir") 

In [ ]:
## 공식으로 제공하는 colab (학습 과정, 로딩과정) 참고
https://colab.research.google.com/drive/1jCkpikz0J2o20FBQmYmAGdiKmJGOMo-o?usp=sharing#scrollTo=otj46qRbtpnd

In [ ]:
## LoRA로 학습된 모델 로딩 예시
from transformers import AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)

In [ ]:
## peft 활용한 모델은 loading에 아래부분 추가되는 듯함
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

device = torch.device('cpu') # cuda
model = model.to(device)
model.eval()
inputs = tokenizer("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label :", return_tensors="pt")

import torch
with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

In [ ]:
## 양자화: https://huggingface.co/docs/transformers/main/en/main_classes/quantization#finetune-a-model-that-has-been-loaded-in-8bit
## 좀 더 자세히 알아보고 싶다면? https://github.com/huggingface/peft